In [3]:
import numpy as np 
import scipy.integrate as integ
from IPython.display import Math
import matplotlib.pyplot as plt


\begin{equation}
f'(x)  = -3x
\end{equation}



In [4]:
def test(x, t):
    return -3*x[0]

In [5]:
#initial conditions
x0 = [1]
t = np.linspace(0, 4, 9)

In [6]:
t

array([ 0. ,  0.5,  1. ,  1.5,  2. ,  2.5,  3. ,  3.5,  4. ])

In [7]:
sol = integ.odeint(test, x0, t)

In [8]:
fig1 = plt.figure()
ax1 = fig1.add_subplot(111)
ax1.plot(t, sol)
plt.show()

\begin{equation}
mx''(t) + cx'(t) + kx(t) = 0\\
\downarrow\\
x_1'(t) = x_2(t)\\
x_2'(t) = -\frac{c}{m}\ x_2(t) - \frac{k}{m}\ x_1(t)
\end{equation}


In [9]:
def vib1(x, t):
    m = 3
    c = 1
    k = 2
    
    return [x[1], -(c/m)*x[1]-(k/m)*x[0]]

\begin{equation}
x(t) = 0\\
x'(t) = 0.25\\
0 \leq t \leq 20
\end{equation}

In [10]:
x0 = [0, 0.25]
t = np.linspace(0, 20, 1000)

In [11]:
solvib = integ.odeint(vib1, x0, t)

In [12]:
fig2 = plt.figure()
ax2 = fig2.add_subplot(111)
ax2.plot(t, solvib)
ax2.set_xlabel('Time')
ax2.set_ylabel('Displacement\nVelocity')
leg2 = ax2.legend(['Displacement','Velocity'])
leg2.get_frame().set_linewidth(0.0)
plt.show()

\begin{equation}
\theta''(t) + \frac{g}{l}sin(\theta(t)) = 0\\
\downarrow\\
\theta_1'(t) = \theta_2(t)\\
\theta_2'(t) = -\frac{g}{l}\ sin(\theta_1(t))
\end{equation}

In [13]:
def pendulum(o, t):
    g = 9.81
    l = 1
    
    return [o[1], -(g/l)*np.sin(o[0])]

In [14]:
o0 = [35*np.pi/180, 0]

In [15]:
solpend = integ.odeint(pendulum, o0, t)

\begin{equation}
\theta(t) = Asin(\omega_nt+\phi)\\
\downarrow\\
A = \sqrt{\frac{w_n^2 x_0^2 + v_0^2}{w_n^2}}\\
\phi = arctan(\frac{w_n x_0}{v_0})
\end{equation}

In [16]:
#Solução analítica
g = 9.81
l = 1
wn = (g/l)**0.5
A = (((wn**2)*(o0[0]**2)+(o0[1]**2))/(wn**2))**0.5
if o0[1] == 0:
    phi = np.pi/2
else:
    phi = np.arctan(wn*o0[0]/o0[1])
oa = A*np.sin(wn*t + phi)

In [17]:
solpendan = A*np.sin(wn*t+phi)

In [18]:
fig3 = plt.figure()
ax3 = fig3.add_subplot(111)
ax3.plot(t, solpendan)
ax3.plot(t, solpend[:,0])
ax3.set_xlabel('Time')
ax3.set_ylabel('Displacement\nVelocity')
leg3 = ax3.legend(['Linear','Non linear'])
leg3.get_frame().set_linewidth(0.0)
plt.show()

In [19]:
class VibSystem(object):
    
    def __init__(self, m, c, k, t, x0):
        self.m = m
        self.c = c
        self.k = k
        self.t = t
        self.x0 = x0
        
    def response(self, x, t):
        m = self.m
        c = self.c
        k = self.k
        return [x[1], -(c/m)*x[1]-(k/m)*x[0]]
    
    def solvib2(self):        
        return integ.odeint(self.response, self.x0, self.t)
    
    def wn(self):
        return (self.k/self.m)**0.5
    
    def damp_ratio(self):
        return self.c/(2*(self.k*self.m)**0.5)
    
    def wd(self):
        return self.wn()*(1-self.damp_ratio()**2)**0.5
    
    def period(self):
        return 1/self.wd()
    
    def amp(self):
        x0 = self.x0[0]
        v0 = self.x0[1]
        zeta = self.damp_ratio()
        wn = self.wn()
        wd = self.wd()
        
        return (((v0+zeta*x0*wn)**2+(x0*wd)**2)**0.5)/wd
    
    def phi(self):
        x0 = self.x0[0]
        v0 = self.x0[1]
        zeta = self.damp_ratio()
        wn = self.wn()
        wd = self.wd()
    
        return np.arctan((x0*wd)/(v0+zeta*wn*x0))
    
    #Plots
    
    def plot_disp(self):
        fig1 = plt.figure()
        ax1 = fig1.add_subplot(111)
        ax1.plot(self.t, self.solvib2()[:,0])
        ax1.set_xlabel('Time')
        ax1.set_ylabel('Displacement')
        leg1 = ax1.legend(['Displacement'])
        leg1.get_frame().set_linewidth(0.0)
        plt.show()
        
    def plot_vel(self):
        fig2 = plt.figure()
        ax2 = fig2.add_subplot(111)
        ax2.plot(self.t, self.solvib2()[:,1])
        ax2.set_xlabel('Time')
        ax2.set_ylabel('Velocity')
        leg2 = ax2.legend(['Velocity'])
        leg2.get_frame().set_linewidth(0.0)
        plt.show()
        
    def plot_disp_vel(self):
        fig3 = plt.figure()
        ax3 = fig3.add_subplot(111)
        ax3.plot(self.t, self.solvib2())
        ax3.set_xlabel('Time')
        ax3.set_ylabel('Displacement\nVelocity')
        leg3 = ax3.legend(['Displacement','Velocity'])
        leg3.get_frame().set_linewidth(0.0)
        plt.show()
        
        

In [20]:
Vib1 = VibSystem(3, 2, 2, t, x0)

In [21]:
Vib1.plot_disp_vel()

In [22]:
Vib117 = VibSystem(1361, 3.81*10**3, 2.688*10**5, t, [0, 0.01/1000])

In [23]:
Vib117.wd()

13.98365687946418

In [24]:
Vib117.damp_ratio()

0.09959815311550789

In [25]:
Vib117.amp()

7.151205214914553e-07

In [26]:
vib131 = VibSystem(49.2*10**-3, 0.11, 857.8, t, [0.1,0])

In [27]:
vib131.damp_ratio()

0.008466172579766066

In [28]:
vib141 = VibSystem(10, 3, 1000, t, [0.01,0])

In [29]:
vib141.damp_ratio()

0.015

In [30]:
vib141.amp()

0.010001125189879353

In [31]:
vib141.phi()

1.5557957642379359

In [32]:
class VibSystemForced(object):
    
    def __init__(self, m, c, k, t, x0, F, w_f, phi_f=0):
        self.m = m
        self.c = c
        self.k = k
        self.t = t
        self.x0 = x0
        self.F = F
        self.w_f = w_f
        
    def response(self, x, t):
        m = self.m
        c = self.c
        k = self.k
        F = self.F
        w_f = self.w_f #frequência de forçamento
        return [x[1], (F*np.cos(w_f*t+phi_f)/m)-(c/m)*x[1]-(k/m)*x[0]]
    
    def solvib2(self):        
        return integ.odeint(self.response, self.x0, self.t)
    
    def wn(self):
        return (self.k/self.m)**0.5
    
    def damp_ratio(self):
        return self.c/(2*(self.k*self.m)**0.5)
    
    def log_dec(self):
        return 2*np.pi*self.damp_ratio()/((1-self.damp_ratio()**2)**0.5)
    
    def wd(self):
        return self.wn()*(1-self.damp_ratio()**2)**0.5
    
    def period(self):
        return 1/self.wd()
    
    def amp(self):
        x0 = self.x0[0]
        v0 = self.x0[1]
        zeta = self.damp_ratio()
        wn = self.wn()
        wd = self.wd()
        
        return (((v0+zeta*x0*wn)**2+(x0*wd)**2)**0.5)/wd
    
    def phi(self):
        x0 = self.x0[0]
        v0 = self.x0[1]
        zeta = self.damp_ratio()
        wn = self.wn()
        wd = self.wd()
    
        return np.arctan((x0*wd)/(v0+zeta*wn*x0))
    
    #Plots
    
    def plot_disp(self):
        fig1 = plt.figure()
        ax1 = fig1.add_subplot(111)
        ax1.plot(self.t, self.solvib2()[:,0])
        ax1.set_xlabel('Time')
        ax1.set_ylabel('Displacement')
        leg1 = ax1.legend(['Displacement'])
        leg1.get_frame().set_linewidth(0.0)
        plt.show()
        
    def plot_vel(self):
        fig2 = plt.figure()
        ax2 = fig2.add_subplot(111)
        ax2.plot(self.t, self.solvib2()[:,1])
        ax2.set_xlabel('Time')
        ax2.set_ylabel('Velocity')
        leg2 = ax2.legend(['Velocity'])
        leg2.get_frame().set_linewidth(0.0)
        plt.show()
        
    def plot_disp_vel(self):
        fig3 = plt.figure()
        ax3 = fig3.add_subplot(111)
        ax3.plot(self.t, self.solvib2())
        ax3.set_xlabel('Time')
        ax3.set_ylabel('Displacement\nVelocity')
        leg3 = ax3.legend(['Displacement','Velocity'])
        leg3.get_frame().set_linewidth(0.0)
        plt.show()
        
        

In [33]:
vib_ex211 = VibSystemForced(10, 1, 1000, t, [0, 0.2], 23, 10.1)

In [34]:
vib_ex211.log_dec()

0.031416319242342894

In [35]:
vib_ex211.amp()

0.020000250004687598

In [36]:
vib_ex211.wn()

10.0

In [37]:
0.1/((4*np.pi**2)+(0.1**2))**0.5

0.015913478971147695

In [38]:
0.2/((4*np.pi**2)+(0.2**2))**0.5

0.03181487509493862

In [39]:
2*np.pi

6.283185307179586